In [197]:
import os, sys
import pandas as pd
import numpy as np
# Display settings: do not abbreviate DataFrame output
pd.set_option("display.max_rows", None)        # show all rows
pd.set_option("display.max_columns", None)     # show all columns
pd.set_option("display.width", None)           # don't wrap to fit console width
pd.set_option("display.max_colwidth", None)    # don't truncate column contents



In [295]:
from pathlib import Path
import pandas as pd
import re

base = Path("responses/cancer")

for csv_path in sorted(base.glob("responses*.csv")):
    print(f"Processing {csv_path} ...")

    df = pd.read_csv(csv_path)

    # Only deduplicate files that actually look like model outputs
    if "prompt" not in df.columns or "raw_response" not in df.columns:
        print("  -> Skipping (missing 'prompt' or 'raw_response').")
        continue

    # --- robust valid_flag ---
    if "valid" in df.columns:
        # Turn into strings, strip, and treat a small set as "true"
        vstr = df["valid"].astype(str).str.strip()
        df["valid_flag"] = vstr.isin(["1", "true", "True", "TRUE", "yes", "Yes"])
    else:
        df["valid_flag"] = False

    df["valid_flag"] = df["valid_flag"].astype(int)

    # raw_response is "usable" if non-null and not just whitespace
    df["raw_ok"] = df["raw_response"].notna() & df["raw_response"].astype(str).str.strip().ne("")

    # Priority: 3 = valid & raw_ok, 2 = valid only, 1 = raw_ok only, 0 = neither
    df["priority"] = df["valid_flag"] * 2 + df["raw_ok"].astype(int)

    before = len(df)

    # Sort so best row per prompt comes first, then drop duplicates on prompt
    df_sorted = df.sort_values(["prompt", "priority"], ascending=[True, False])
    df_nodup = df_sorted.drop_duplicates(subset=["prompt"], keep="first")

    # Drop helper columns
    df_nodup = df_nodup.drop(columns=["valid_flag", "raw_ok", "priority"])

    after = len(df_nodup)
    print(f"  -> {before} rows -> {after} rows after dedup")

    # Overwrite original file
    df_nodup.to_csv(csv_path, index=False)


Processing responses/cancer/responses_obs0_int3_shuf3_anon_Llama-3.2-3B.csv ...
  -> 30 rows -> 30 rows after dedup
Processing responses/cancer/responses_obs0_int3_shuf3_anon_Qwen3-30B-A3B-Instruct-2507.csv ...
  -> 30 rows -> 30 rows after dedup
Processing responses/cancer/responses_obs0_int3_shuf3_anon_Qwen3-32B.csv ...
  -> 30 rows -> 30 rows after dedup
Processing responses/cancer/responses_obs0_int3_shuf3_anon_Qwen3-4B.csv ...
  -> 21 rows -> 21 rows after dedup
Processing responses/cancer/responses_obs0_int3_shuf3_anon_gemini-2.5-flash.csv ...
  -> 30 rows -> 30 rows after dedup
Processing responses/cancer/responses_obs0_int3_shuf3_anon_gpt-4o-mini.csv ...
  -> 30 rows -> 30 rows after dedup
Processing responses/cancer/responses_obs0_int3_shuf3_anon_rules_Qwen3-4B.csv ...
  -> 10 rows -> 10 rows after dedup
Processing responses/cancer/responses_obs0_int3_shuf3_anon_rules_gemini-2.5-flash.csv ...
  -> 30 rows -> 30 rows after dedup
Processing responses/cancer/responses_obs0_int3_s

In [324]:
from pathlib import Path
import csv
import json
import subprocess
import re

BASE_DIR = Path("responses/cancer")
SUMMARY_CSV = BASE_DIR / "eval_summary.csv"

def parse_given_edges_tag(path: Path):
    """
    Parse given-edges info from filenames like:
      ..._gedge20_...
    Returns (has_given_edges, given_edge_frac, given_edge_pct)
      has_given_edges: 0/1
      given_edge_frac: float or None
      given_edge_pct:  int or None
    """
    stem = path.stem  # e.g. responses_obs200_int3_shuf3_gedge20_anon_gpt-4o-mini
    m = re.search(r"_gedge(\d+)", stem)
    if not m:
        return 0, None, None
    pct = int(m.group(1))
    frac = pct / 100.0
    return 1, frac, pct

def count_nonempty(colname, rows):
    return sum(1 for r in rows if (r.get(colname) or "").strip())

def count_valid_flag(rows):
    # valid column is expected to be 1/0 or truthy/falsy
    return sum(1 for r in rows if str(r.get("valid", "")).strip() in {"1", "true", "True"})

def count_error_raw(rows):
    # count rows where raw_response contains "[ERROR]"
    return sum(
        1
        for r in rows
        if "[ERROR]" in (r.get("raw_response") or "")
    )

incomplete_files = []
complete_files = []
file_stats = {}  # file -> dict with n_rows, n_raw, n_valid, n_error

if not BASE_DIR.exists():
    print(f"Base directory not found: {BASE_DIR.resolve()}")
else:
    for csv_path in sorted(BASE_DIR.rglob("*.csv")):
        try:
            with csv_path.open("r", encoding="utf-8", newline="") as f:
                reader = csv.DictReader(f)
                rows = list(reader)
                fieldnames = reader.fieldnames or []
        except Exception as e:
            print(f"[ERROR] Failed to read {csv_path}: {e}")
            incomplete_files.append((csv_path, "read_error"))
            continue

        n_rows = len(rows)
        has_prompt = "prompt" in fieldnames
        has_raw = "raw_response" in fieldnames

        # --------- SPECIAL CASE: ENCO prediction files ---------
        # e.g. predictions_obs200_int3_ENCO.csv
        is_enco = "ENCO" in csv_path.name

        if is_enco:
            # ENCO files don't have prompt/raw_response; treat them as
            # complete as long as they have at least 1 row.
            n_prompt = 0
            n_raw = 0
            n_valid = 0
            n_error = 0

            complete = n_rows > 0
            status = "OK" if complete else "INCOMPLETE"
            print(
                f"{csv_path} -> rows={n_rows}, (ENCO: skipping prompt/raw checks) "
                f"valid_flags={n_valid} [{status}]"
            )

        else:
            # --------- Normal LLM response files (must have prompt & raw_response) ---------
            if not has_prompt or not has_raw:
                reason_parts = []
                if not has_prompt:
                    reason_parts.append("missing prompt column")
                if not has_raw:
                    reason_parts.append("missing raw_response column")
                reason = "; ".join(reason_parts)
                print(f"{csv_path}: INCOMPLETE ({reason})")
                incomplete_files.append((csv_path, reason))
                continue

            n_prompt = count_nonempty("prompt", rows)
            n_raw = count_nonempty("raw_response", rows)
            n_valid = count_valid_flag(rows) if "valid" in fieldnames else 0
            n_error = count_error_raw(rows)

            complete = (n_prompt == n_raw == n_rows)
            status = "OK" if complete else "INCOMPLETE"
            print(
                f"{csv_path} -> rows={n_rows}, prompts={n_prompt}, "
                f"raw_responses={n_raw}, error_responses={n_error}, "
                f"valid_flags={n_valid} [{status}]"
            )

        # --------- Store stats for later use in the summary CSV ---------
        file_stats[csv_path] = {
            "n_rows": n_rows,
            "completed_rows": n_raw,
            "valid_flag_rows": n_valid,
            "error_raw_responses": n_error,
        }

        if complete:
            complete_files.append(csv_path)
        else:
            reason = f"rows={n_rows}, prompts={n_prompt}, raw_responses={n_raw}"
            incomplete_files.append((csv_path, reason))

    # --------- Delete files with only [ERROR] raw_responses ---------
    all_error_files = []
    for csv_path_err, stats in list(file_stats.items()):
        n_rows_err = stats["n_rows"]
        n_error_err = stats["error_raw_responses"]

        # For ENCO, n_error_err == 0, so this will never trigger
        if n_rows_err > 0 and n_rows_err == n_error_err:
            print(f"[CLEANUP] Deleting {csv_path_err} (all {n_rows_err} raw_responses contain '[ERROR]').")
            all_error_files.append(csv_path_err)

            # Remove from complete_files if it was marked complete
            if csv_path_err in complete_files:
                complete_files.remove(csv_path_err)

            # Delete the CSV itself
            try:
                csv_path_err.unlink()
            except FileNotFoundError:
                pass

            # Optionally delete any sidecar files created in previous runs
            jsonl_path = csv_path_err.with_suffix(".jsonl")
            summary_path = csv_path_err.with_suffix(csv_path_err.suffix + ".summary.json")
            for sidecar in (jsonl_path, summary_path):
                try:
                    sidecar.unlink()
                except FileNotFoundError:
                    pass

    # --------- Completeness summary to stdout ---------
    print("\n=== Completeness Summary ===")
    if not incomplete_files:
        print("All CSV files in responses/cancer/ appear complete.")
    else:
        print("These files look incomplete:")
        for path, reason in incomplete_files:
            print(f"- {path}: {reason}")

    # --------- Run evaluate.py on complete files & collect metrics ---------
    summary_rows = []

    for csv_path in complete_files:
        print(f"\nEvaluating {csv_path}")
        proc = subprocess.run(
            ["python", "evaluate.py", "--csv", str(csv_path)],
            capture_output=True,
            text=True,
        )
        if proc.returncode != 0:
            print(f"[ERROR] evaluate.py failed on {csv_path}")
            print(proc.stdout)
            print(proc.stderr)
            continue

        # evaluate.py should have created <file>.summary.json
        summary_path = csv_path.with_suffix(csv_path.suffix + ".summary.json")
        if not summary_path.exists():
            print(f"[WARN] Summary JSON not found for {csv_path}: {summary_path}")
            continue

        try:
            with summary_path.open("r", encoding="utf-8") as f_sum:
                metrics = json.load(f_sum)
        except Exception as e:
            print(f"[ERROR] Failed to read summary JSON for {csv_path}: {e}")
            continue

        # Build one combined row: file name + completeness stats + eval metrics
        # Build one combined row: file name + completeness stats + eval metrics
        base_stats = file_stats[csv_path]

        # Rename potentially confusing keys from evaluate.py
        metrics["eval_num_rows"] = metrics.pop("num_rows", None)
        metrics["eval_valid_rows"] = metrics.pop("valid_rows", None)

        # Decode given-edges info from filename
        has_given_edges, given_edge_frac, given_edge_pct = parse_given_edges_tag(csv_path)

        row = {
            "file": str(csv_path),
            "num_rows": base_stats["n_rows"],
            "completed": base_stats["completed_rows"],        # count(raw_response nonempty)
            "valid": base_stats["valid_flag_rows"],           # from 'valid' column
            "error_raw_responses": base_stats["error_raw_responses"],  # rows with "[ERROR]" in raw_response

            # new columns:
            "given_edges": has_given_edges,       # 0/1 flag
            "given_edge_frac": given_edge_frac,   # e.g. 0.2 for _gedge20
            "given_edge_pct": given_edge_pct,     # e.g. 20 for _gedge20
        }
        row.update(metrics)

        summary_rows.append(row)


    # --------- Write global metrics summary CSV ---------
    if summary_rows:
        SUMMARY_CSV.parent.mkdir(parents=True, exist_ok=True)
        # Collect union of all keys in case some metrics differ
        all_keys = []
        for r in summary_rows:
            for k in r.keys():
                if k not in all_keys:
                    all_keys.append(k)

        with SUMMARY_CSV.open("w", encoding="utf-8", newline="") as f_out:
            writer = csv.DictWriter(f_out, fieldnames=all_keys)
            writer.writeheader()
            for row in summary_rows:
                writer.writerow(row)
        print(f"\nEvaluation summary written to: {SUMMARY_CSV.resolve()}")
    else:
        print("\nNo complete files were successfully evaluated; no summary CSV written.")


responses/cancer/completeness_summary.csv: INCOMPLETE (missing prompt column; missing raw_response column)
responses/cancer/eval_summary.csv: INCOMPLETE (missing prompt column; missing raw_response column)
responses/cancer/predictions_obs0_int3_ENCO.csv -> rows=1, (ENCO: skipping prompt/raw checks) valid_flags=0 [OK]
responses/cancer/predictions_obs2000_int30_ENCO.csv -> rows=1, (ENCO: skipping prompt/raw checks) valid_flags=0 [OK]
responses/cancer/predictions_obs200_int0_ENCO.csv -> rows=1, (ENCO: skipping prompt/raw checks) valid_flags=0 [OK]
responses/cancer/predictions_obs200_int3_ENCO.csv -> rows=1, (ENCO: skipping prompt/raw checks) valid_flags=0 [OK]
responses/cancer/predictions_obs200_int5_ENCO.csv -> rows=1, (ENCO: skipping prompt/raw checks) valid_flags=0 [OK]
responses/cancer/predictions_obs500_int5_ENCO.csv -> rows=1, (ENCO: skipping prompt/raw checks) valid_flags=0 [OK]
responses/cancer/prompts_obs0_int3_shuf3_anon_gemini-2.5-flash.csv -> rows=30, prompts=30, raw_responses

In [328]:
df = pd.read_csv('responses/cancer/eval_summary.csv')
mask = (
    # df['file'].str.contains('steps') &
    # df['file'].str.contains('obs200_int3_shuf3_anon') &
    # df['file'].str.contains('responses') &
    df['file'].str.contains('_anon_rules_steps_gpt', case=False)
    # & ~df['file'].str.contains('anon')
    # & (df['valid'].astype(int) == 0)
)
# df[mask][['file', 'num_rows','error_raw_responses','valid', 'avg_f1', 'avg_shd']].sort_values('avg_f1', ascending=False)
df[mask].sort_values('avg_f1', ascending=False)[["file","num_pred_edges","avg_f1","avg_shd"]]



,file,num_pred_edges,avg_f1,avg_shd
63,responses/cancer/responses_obs200_int3_shuf3_anon_rules_steps_gpt-4o-mini.csv,4.000000,0.614153,3.133333
32,responses/cancer/responses_obs200_int0_shuf3_anon_rules_steps_gpt-4o-mini.csv,3.714286,0.594033,3.285714
71,responses/cancer/responses_obs200_int3_shuf3_gedge20_anon_rules_steps_gpt-4o-mini.csv,2.766667,0.523651,3.166667
16,responses/cancer/responses_obs0_int3_shuf3_anon_rules_steps_gpt-4o-mini.csv,5.793103,0.516652,4.758621
64,responses/cancer/responses_obs200_int3_shuf3_anon_rules_steps_gpt-5-mini-2025-08-07.csv,NaN,NaN,0.000000


In [310]:
import pandas as pd
from pathlib import Path
import math

EVAL_SUMMARY = Path("responses/cancer/eval_summary.csv")

df = pd.read_csv(EVAL_SUMMARY)
df = df[~df['file'].str.contains('4B', case=False) &
        ~df['file'].str.contains('2024-07-18', case=False)]

# --- 0. Keep only anon LLM runs + ENCO baseline --------------------
file_str = df["file"].astype(str)
is_anon = file_str.str.contains("_anon", regex=False)
is_enco = file_str.str.contains("ENCO", regex=False)  # non-anon ENCO runs
df = df[is_anon | is_enco].reset_index(drop=True)

from pathlib import Path

def detect_variant(path_str: str) -> str:
    name = Path(path_str).name.lower()
    if "rules_steps" in name:
        return "rules_steps"
    if "rules" in name:
        return "rules"
    if "steps" in name:
        return "steps"
    return "base"

df["variant"] = df["file"].astype(str).apply(detect_variant)

# Make life easier: ensure 'valid' exists & integer
if "valid" in df.columns:
    df["valid"] = df["valid"].fillna(0).astype(int)
else:
    df["valid"] = 0


# --- 2. Metric lookup helpers --------------------------------------

def lookup_metrics(setting_tags, model_tag,
                   variant=None,
                   min_valid=1,
                   given_edge_count=None):
    """
    setting_tags: list like ["obs200_int3"]
    model_tag:    e.g. "gpt-4o-mini" or "ENCO"
    variant:      one of {"base", "rules", "steps", "rules_steps"} or None
    given_edge_count: if not None, require df['given_edge_count'] == this value
    """
    m = df["file"].astype(str).str.contains(setting_tags[0], regex=False)
    for s in setting_tags[1:]:
        m &= df["file"].astype(str).str.contains(s, regex=False)

    # model name constraint
    m &= df["file"].astype(str).str.contains(model_tag, regex=False)

    # only apply valid filter if requested
    if min_valid > 0:
        m &= df["valid"] >= min_valid

    # variant constraint (base / rules / steps / rules_steps)
    if variant is not None:
        m &= (df["variant"] == variant)

    # given-edge constraint
    if given_edge_count is not None and "given_edge_count" in df.columns:
        m &= (df["given_edge_count"] == given_edge_count)

    subset = df[m]
    if subset.empty:
        return None, None

    row = subset.iloc[0]   # or subset.mean() if you prefer
    return row["avg_f1"], row["avg_shd"]


def fmt(x, ndigits=2):
    if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
        return ""
    return f"{x:.{ndigits}f}"


# -------------------------------------------------------------------
# 3. Settings (Obs/Inter combinations)
# -------------------------------------------------------------------

SETTINGS = {
    "O200_I3": ["obs200_int3"],
    "O200_I0": ["obs200_int0"],
    "O0_I3":   ["obs0_int3"],
}

# -------------------------------------------------------------------
# 4. Extract model tags from filenames (LLMs only; ENCO is special)
# -------------------------------------------------------------------

def extract_model_tag(path_str: str) -> str:
    """Get the model part from a path, e.g.
       '...responses_obs0_int3_shuf3_anon_Qwen3-32B.csv' -> 'Qwen3-32B'
       '...predictions_obs200_int3_ENCO.csv'             -> 'ENCO'
    """
    stem = Path(path_str).stem    # e.g. 'responses_obs0_int3_shuf3_anon_Qwen3-32B'
    parts = stem.split("_")
    return parts[-1]

# Only look at response files for LLM models (ENCO is in predictions files)
mask_llm = df["file"].str.contains("responses", regex=False) & \
           ~df["file"].str.contains("ENCO", regex=False)
df_models = df[mask_llm].copy()
df_models["model"] = df_models["file"].apply(extract_model_tag)

all_models = sorted(df_models["model"].unique())
# print("Detected LLM models:", all_models)

llm_models = all_models  # ENCO handled separately

PRETTY_NAME = {
    "gemini-2.5-flash": "Gemini-2.5-Flash",
    # add others if you care; fall back to tag itself
}
def pretty_model_name(model_tag: str) -> str:
    return PRETTY_NAME.get(model_tag, model_tag)

# -------------------------------------------------------------------
# 5. Variant blocks (sections)
# -------------------------------------------------------------------

VARIANTS = [
    ("base",        r"\multicolumn{7}{l}{\textbf{\textit{Zero-Shot LLMs}}} \\"),
    ("rules",       r"\multicolumn{7}{l}{\textbf{\textit{Zero-Shot LLMs + Causality Rules}}} \\"),
    ("steps",       r"\multicolumn{7}{l}{\textbf{\textit{Zero-Shot LLMs + CD Steps}}} \\"),
    ("rules_steps", r"\multicolumn{7}{l}{\textbf{\textit{Zero-Shot LLMs + Causality Rules + CD Steps}}} \\"),
]

ROWS = []

# 5.a ENCO (pure causal discovery method, non-anon predictions_*.csv)
ROWS.append({
    "section": r"\multicolumn{7}{l}{\textbf{\textit{Pure Causal Discovery Method}}} \\",
    "label":   "ENCO",
    "model_tag": "ENCO",
    "variant": "base",
    "given_edge_count": None,   # no given-edge prior
})

# 5.b LLM rows for each variant + model
for variant_key, section_header in VARIANTS:
    first_in_section = True
    for model_tag in llm_models:
        ROWS.append({
            "section":   section_header if first_in_section else None,
            "label":     pretty_model_name(model_tag),
            "model_tag": model_tag,
            "variant":   variant_key,
            "given_edge_count": None,   # normal zero-shot / rules / steps runs
        })
        first_in_section = False

# 5.c NEW: Zero-Shot LLMs + Given One Edge
# pick models that have given_edge_count == 1 somewhere
if "given_edge_count" in df.columns:
    mask_given = (
        (df["given_edge_count"] == 1) &
        df["file"].str.contains("responses", regex=False) &
        ~df["file"].str.contains("ENCO", regex=False)
    )
    df_given = df[mask_given].copy()
    given_models = sorted(df_given["file"].apply(extract_model_tag).unique())
else:
    given_models = []

first_in_section = True
for model_tag in given_models:
    ROWS.append({
        "section": r"\multicolumn{7}{l}{\textbf{\textit{Zero-Shot LLMs + Given One Edge}}} \\"
                   if first_in_section else None,
        "label":   pretty_model_name(model_tag),
        "model_tag": model_tag,
        "variant": "base",          # assume these are base + given-edge
        "given_edge_count": 1,      # key for filtering in lookup_metrics
    })
    first_in_section = False

# -------------------------------------------------------------------
# 6. Build LaTeX lines (F1 / SHD table)
# -------------------------------------------------------------------
lines = []

lines.append(r"\begin{table*}[ht!]")
lines.append(r"\centering")
lines.append(r"\setlength{\tabcolsep}{6pt}")
lines.append(r"\caption{Causal discovery performance on the \textit{Cancer} causal graph. The variable names are anonymized.}")
lines.append(r"\resizebox{\textwidth}{!}{%")
lines.append(r"\begin{tabular}{lcccccc}")
lines.append(r"\toprule")
lines.append(
    r" & \multicolumn{2}{c}{\textbf{Obs. 200, Inter. 3}}"
    r" & \multicolumn{2}{c}{\textbf{Obs. 200, Inter. 0}}"
    r" & \multicolumn{2}{c}{\textbf{Obs. 0, Inter. 3}} \\"
)
lines.append(r"\cmidrule(lr){2-3}\cmidrule(lr){4-5}\cmidrule(lr){6-7}")
lines.append(
    r"\textbf{Method} & \textbf{SHD}~$\downarrow$ & \textbf{F1}~$\uparrow$"
    r" & \textbf{SHD}~$\downarrow$ & \textbf{F1}~$\uparrow$"
    r" & \textbf{SHD}~$\downarrow$ & \textbf{F1}~$\uparrow$ \\"
)

lines.append(r"\midrule")

for rowdef in ROWS:
    # Optional section header
    if rowdef.get("section"):
        lines.append(rowdef["section"])

    label       = rowdef["label"]
    model_tag   = rowdef["model_tag"]
    variant_key = rowdef["variant"]
    given_edge_count = rowdef.get("given_edge_count", None)

    # ENCO: don't require valid>=1, and ignore variant filter
    is_enco = (model_tag == "ENCO")
    min_valid = 0 if is_enco else 1
    variant_for_lookup = None if is_enco else variant_key

    cells = []
    for key, setting_subs in SETTINGS.items():
        f1, shd = lookup_metrics(
            setting_subs,
            model_tag=model_tag,
            variant=variant_for_lookup,
            min_valid=min_valid,
            given_edge_count=given_edge_count,
        )
        cells.append(fmt(f1))
        cells.append(fmt(shd, ndigits=2))

    # If *all* entries for this row are empty, skip the row entirely
    if all(c == "" for c in cells):
        continue

    row_tex = (
        f"{label} & {cells[1]} & {cells[0]} & "
        f"{cells[3]} & {cells[2]} & {cells[5]} & {cells[4]} \\\\"
    )

    lines.append(row_tex)
    lines.append(r"\addlinespace[0.8ex]")

lines.append(r"\bottomrule")
lines.append(r"\end{tabular}")
lines.append(r"}")
lines.append(r"\end{table*}")

latex_table = "\n".join(lines)
print(latex_table)


# -------------------------------------------------------------------
# 7. Valid-ratio table, same sections (including Given One Edge)
# -------------------------------------------------------------------

def lookup_valid_counts(setting_tags, model_tag,
                        variant=None,
                        min_rows=1,
                        given_edge_count=None):
    """
    Returns (valid, num_rows) or (None, None) if not found.
    """
    m = df["file"].astype(str).str.contains(setting_tags[0], regex=False)
    for s in setting_tags[1:]:
        m &= df["file"].astype(str).str.contains(s, regex=False)

    # constrain to this model
    m &= df["file"].astype(str).str.contains(model_tag, regex=False)

    # constrain to variant if given
    if variant is not None:
        m &= (df["variant"] == variant)

    # constrain to given_edge_count if given
    if given_edge_count is not None and "given_edge_count" in df.columns:
        m &= (df["given_edge_count"] == given_edge_count)

    subset = df[m]
    if subset.empty:
        return None, None

    row = subset.iloc[0]
    num_rows = row.get("num_rows", 0)
    valid    = row.get("valid", 0)

    if not num_rows or num_rows < min_rows:
        return None, None

    return int(valid), int(num_rows)


valid_lines = []

valid_lines.append(r"\begin{table*}[ht!]")
valid_lines.append(r"\centering")
valid_lines.append(r"\setlength{\tabcolsep}{6pt}")
valid_lines.append(
    r"\caption{Valid adjacency extraction ratio for each method and data setting on the \textit{Cancer} graph.}"
)
valid_lines.append(r"\resizebox{\textwidth}{!}{%")
valid_lines.append(r"\begin{tabular}{lccc}")
valid_lines.append(r"\toprule")
valid_lines.append(
    r" & \textbf{Obs. 200, Inter. 3}"
    r" & \textbf{Obs. 200, Inter. 0}"
    r" & \textbf{Obs. 0, Inter. 3} \\"
)
valid_lines.append(r"\midrule")

for rowdef in ROWS:
    # Optional section header – adapt from 7 columns to 4
    if rowdef.get("section"):
        sec = rowdef["section"]
        sec = sec.replace(r"\multicolumn{7}", r"\multicolumn{4}")
        valid_lines.append(sec)

    label       = rowdef["label"]
    model_tag   = rowdef["model_tag"]
    variant_key = rowdef["variant"]
    given_edge_count = rowdef.get("given_edge_count", None)

    cells = []
    for key, setting_subs in SETTINGS.items():
        valid_count, num_rows = lookup_valid_counts(
            setting_subs,
            model_tag=model_tag,
            variant=variant_key if model_tag != "ENCO" else None,
            given_edge_count=given_edge_count,
        )

        if valid_count is None or num_rows is None:
            cells.append("")  # empty cell
        else:
            cells.append(rf"$\frac{{{valid_count}}}{{{num_rows}}}$")

    # Skip row entirely if ALL settings are empty
    if all(c == "" for c in cells):
        continue

    row_tex = f"{label} & {cells[0]} & {cells[1]} & {cells[2]} \\\\"
    valid_lines.append(row_tex)
    valid_lines.append(r"\addlinespace[0.8ex]")

valid_lines.append(r"\bottomrule")
valid_lines.append(r"\end{tabular}")
valid_lines.append(r"}")
valid_lines.append(r"\end{table*}")

latex_valid_table = "\n".join(valid_lines)
print(latex_valid_table)


\begin{table*}[ht!]
\centering
\setlength{\tabcolsep}{6pt}
\caption{Causal discovery performance on the \textit{Cancer} causal graph. The variable names are anonymized.}
\resizebox{\textwidth}{!}{%
\begin{tabular}{lcccccc}
\toprule
 & \multicolumn{2}{c}{\textbf{Obs. 200, Inter. 3}} & \multicolumn{2}{c}{\textbf{Obs. 200, Inter. 0}} & \multicolumn{2}{c}{\textbf{Obs. 0, Inter. 3}} \\
\cmidrule(lr){2-3}\cmidrule(lr){4-5}\cmidrule(lr){6-7}
\textbf{Method} & \textbf{SHD}~$\downarrow$ & \textbf{F1}~$\uparrow$ & \textbf{SHD}~$\downarrow$ & \textbf{F1}~$\uparrow$ & \textbf{SHD}~$\downarrow$ & \textbf{F1}~$\uparrow$ \\
\midrule
\multicolumn{7}{l}{\textbf{\textit{Pure Causal Discovery Method}}} \\
ENCO & 7.00 &  & 4.00 &  & 9.00 & 0.18 \\
\addlinespace[0.8ex]
\multicolumn{7}{l}{\textbf{\textit{Zero-Shot LLMs}}} \\
Qwen3-30B-A3B-Instruct-2507 & 3.60 & 0.63 &  &  & 4.37 & 0.69 \\
\addlinespace[0.8ex]
Qwen3-32B & 5.20 & 0.39 & 4.00 & 0.53 & 4.52 & 0.52 \\
\addlinespace[0.8ex]
Gemini-2.5-Flash & 5.30

In [319]:
from copy import deepcopy
from pathlib import Path
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
sys.path.append("../")
from causal_graphs.graph_real_world import load_graph_file
from causal_graphs.graph_visualization import visualize_graph
import matplotlib.pyplot as plt
# ---------------------------------------------------------------------
# 1. Load the cancer graph
# ---------------------------------------------------------------------
bif_path = Path("../causal_graphs/real_data/small_graphs/cancer.bif")
graph = load_graph_file(str(bif_path))

# ---------------------------------------------------------------------
# 2. Build a NetworkX graph with integer nodes (0..n-1)
#    so we can reuse the same layout but change labels.
# ---------------------------------------------------------------------
G = nx.DiGraph()
n = len(graph.variables)

# Nodes are integers, but we keep two separate label dicts:
labels_orig = {}
labels_anon = {}

for i, v in enumerate(graph.variables):
    G.add_node(i)
    labels_orig[i] = v.name        # original variable name
    labels_anon[i] = f"X{i+1}"     # anonymized name

# Add directed edges from the CausalDAG
for (u_idx, v_idx) in graph.edges.tolist():
    G.add_edge(u_idx, v_idx)

# ---------------------------------------------------------------------
# 3. Compute a single layout (Graphviz) and reuse for both panels
# ---------------------------------------------------------------------
pos = graphviz_layout(G, prog="dot")  # requires graphviz + pydot

# ---------------------------------------------------------------------
# 4. Plot side-by-side: original (left) and anonymized (right)
# ---------------------------------------------------------------------
figsize = max(3, n ** 0.7)
fig, axes = plt.subplots(1, 2, figsize=(2 * figsize, figsize))

# Common draw kwargs to mimic your existing style
draw_kwargs = dict(
    arrows=True,
    node_color="lightgrey",
    edgecolors="black",
    node_size=600,
    arrowstyle="-|>",
    arrowsize=16,
)

# Left: original names
ax = axes[1]
nx.draw(G, pos, ax=ax, with_labels=False, **draw_kwargs)
nx.draw_networkx_labels(G, pos, labels=labels_orig, font_weight="bold", ax=ax)
ax.set_title("Cancer graph (original names)")
ax.set_axis_off()
ax.margins(0.2)  # <<< add padding inside axes
# Right: anonymized names
ax = axes[0]
nx.draw(G, pos, ax=ax, with_labels=False, **draw_kwargs)
nx.draw_networkx_labels(G, pos, labels=labels_anon, font_weight="bold", ax=ax)
ax.set_title("Cancer graph (anonymized)")
ax.set_axis_off()
ax.margins(0.2)
plt.tight_layout(pad=1.0)

plt.tight_layout()
fig.savefig("cancer_graph_original_vs_anon.pdf", bbox_inches="tight", transparent=True)
plt.close(fig)

In [337]:
df_check = pd.read_csv("responses/cancer/responses_obs200_int3_shuf3_anon_Qwen3-32B.csv")

In [338]:
df_check['prediction'].nunique()

10